In [3]:
import json

In [4]:
# Opening JSON file 
with open('DEEPre_Sequences.json') as json_file: 
    data = json.load(json_file) 

In [5]:
data

{'Q1LU25': 'MLNNILKIATRQSPLAIWQANYVRNQLLSFYPTLLIELVPIVTSGDLILDKPLMKAGGKRLFIKELEQAMLERRADIAVHSMKDITISFPEGIGLAVLCAREDPRDAFISTHYASIDLLPTGAVVGTSSMRRQCQLRERRPDLVMRDLRGNIGTRLEKLDKGEYDALILAAAGLKRLDLAHRIRMIIDPTELLPAVGQGVIGIEYRLEDTHILSILAPLHHSATALRVSAERAMNAKLAGGCQVPIGSYAEIEGDQIWLRALVGSPDGSLIIRSEGRAPLSQAEILGQSIANDLLYRGAESILCRAFQVDS',
 'Q7VRM4': 'MQAKILRIATRKSPLAICQACYVCNKLKHYHPHIQTELIPIITTGDTSLHTVSKTNKIKKGAFIKELEYALIEHRADIAVHSMKDVTVLLPKELIIPVLCKRHDPRDAFVSIQYPNIHSLPTGSIIGTSSLRRQCQIRAQRSDLILHDLRGNINTRIKKLHQGQYNAIILAVAGLTRLKLTKYIQTYIDPSELLPAMGQGVIAIECRINDINTISLLSPLYHKETSLCIKSERAVTTYLESYCRLPIASYAEIQNNQIWLRALIGLPDGSIIIRTEGIASLDKAEELGLNLAKDLLIQFKKNTHFDINSIK',
 'Q7WKM1': 'MAAVERLTIATRASRLALWQAEHVRDLLRARYPACSVELLTLTTRGDQILDRTLSKVGGKGLFVKELETALLDGRADLAVHSLKDVPVDLHAPFELSCVLERADPRDAFVSNDYGSLADLPPGAAVGTSSLRRESQIRARYPHLVVKPLRGNLDTRLGKLDNGDYAAIVLAAAGLERLGLAARIRALLEPADSLPAAGQGALGIEILQGRADVRAMLAPLGDAATLACVTAERAVSRMLGGSCQVPLAAYARIDGDELALRALVAAPDGRRIVRAERRGPRDQAQAIGESAARDLLADGADAILAELLPTSPAP',
 'Q65GK1': 'MRNIIVG

In [7]:
import torch
import esm

# Load ESM-1b model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()


In [8]:
data_list = []

max_count = 1000
count = 0
for id in data:
    data_list.append((id, data[id]))
    


ch_labels, batch_strs, batch_tokens = batch_converter(data_list[0:1000])

In [10]:
len(ch_labels)

1000

In [ ]:
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data_list[0:1000]):
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), attention_contacts in zip(data_list[0:1000], results["contacts"]):
    plt.matshow(attention_contacts[: len(seq), : len(seq)])
    plt.title(seq)
    plt.show()